## Predição de Fraudes "Card Not Present" (IEEE-CIS Dataset)

O problema escolhido para realização do projeto foi de fraudes em transações sem cartão presencial, conhecidas como **"Card Not Present Fraud"**. Os dados foram preparados e disponibilizados pela IEEE Computational Intelligence Society e lançados durante uma competição [IEEE-CIS Fraud Detection](https://www.kaggle.com/competitions/ieee-fraud-detection/overview) no Kaggle.

O que torna esse projeto mais interessante é que os dados são de **transações reais** foram fornecidos pela [Vesta Corporation](https://vesta.io/). A Vesta é uma empresa especializada em soluções de proteção contra fraudes e processamento de pagamentos para transações móveis e online. A empresa utiliza modelos avançados de *machine learning* para analisar mais de US$ 4 bilhões em transações anualmente, fornecendo serviços que permitem a aprovação de vendas em milissegundos e o processamento de pagamentos em mais de 40 países.

O conjunto de dados possui cerca de 600.000 registros e contém mais de 430 características. Portanto, além do desafio de modelagem para predição de fraudes, é preciso considerar o grande volume de dados e a alta dimensionalidade. Por serem transações reais, um grande número de *features* foram anonimizadas, garantindo a privacidade dos clientes, portanto, não é possível saber qual o conteúdo real de todas as variáveis.

### Definição do problema

Imagine a seguinte situação. Você compra um café da manhã na padaria e se dirige ao caixa para pagar. Como é muito comum nos tempos atuais, você utiliza seu telefone para pagar, usando um cartão digital por aproximação, mas sua compra é negada. Ou você decide aproveitar as promoções de fim de ano em determinada plataforma de e-commerce, preenche as informações do seu cartão, mas sua compra não pode ser efetuada.

Embora você tenha certeza que possui dinheiro suficiente para realizar a compra, por algum motivo sua transação não é efetuada. Isso acontece por que, todos os anos, mais de 30 bilhões de dólares são movimentados em transações fraudulentas de cartões de crédito e as empresas envolvidas nesses serviços, como bancos, instituições de pagamento e prestadoras de serviço, investem pesado em **sistemas de prevenção de fraude**. Apesar de nem sempre serem acertivos, como no caso descrito acima quando houve uma falso positivo, esses sistema podem evitar uma grande dor de cabeça para os clientes e para essas instituições.

A fraude em transações "Card Not Present" (CNP) ocorre quando compras são realizadas sem a presença física do cartão. Com o avanço da tecnologia e a popularização das compras pela internet, esse tipo de transação tornou-se predominante. No Brasil, por exemplo, 61% dos consumidores preferem comprar online em vez de em lojas físicas, e 78% realizam pelo menos uma compra mensal pela internet. Nessas compras, os consumidores inserem os dados do cartão, como número, data de validade e código de segurança, ou utilizam carteiras digitais em dispositivos móveis, facilitando as transações, mas também aumentando os riscos de fraude.

Considerando esse cenário, o objetivo do trabalho é criar um **modelo de classificação** capaz de estimar a probabilidade das trações serem verdadeiras ou fraudes. Em paralelo, também discutir conceitos importantes desse tipo de problema, como classes desbalanceadas, redução de dimensionalidade e uso adequado de métricas.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, precision_score, recall_score, f1_score

# sklearn models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import RUSBoostClassifier

from lightgbm import LGBMClassifier

### Análise Exploratória de Dados

Como discutido, neste problema vamos criar um classificador para prever a probabilidade de uma transação ser fraudulenta, indicada pelo alvo binário `isFraud`. Os dados de treinamento do Kaggle estão divididos em dois arquivos, `train_identity.csv` e `train_transaction.csv`, que são unidos pela chave `TransactionID`.

Por ser uma competição, o Kaggle não permite baixar os arquivos sem fazer login, por isso, não foi possível ler os dados através da URL. Foi necessário baixar os arquivos, que podem ser encontrados na pasta `/data` e ler localmente. 

#### Identity

In [ ]:
train_identity = pd.read_csv('../data/train_identity.csv')
train_identity.shape

In [ ]:
train_identity.head(5)

In [ ]:
identity_columns = train_identity.columns.difference(['TransactionID'])

#### Transaction

Por causa do grande volume de *features* nos dados de transações (394), vamos ler os dados em *chunks* ou blocos de 100.000 registros, o que otimiza a alocação de memória da máquina. Após concatenar os blocos em um único DataFrame, podemos deletar esses blocos para economizar recursos.

In [ ]:
transaction_chunks = pd.read_csv('../data/train_transaction.csv', chunksize=10 ** 5)
train_transaction = pd.concat(transaction_chunks)

train_transaction.shape

In [ ]:
train_transaction.head(5)

In [ ]:
del transaction_chunks

In [ ]:
associated = np.sum(train_transaction['TransactionID'].isin(train_identity['TransactionID'].unique()))
total_transaction = train_transaction.shape[0]

pct_records = np.divide(associated, total_transaction) * 100

print(f'{pct_records:.2f}% do registro em Transaction ({total_transaction}) possuem registros de Identity associados.')

#### isFraud

In [ ]:
train_transaction['isFraud'].value_counts(normalize=True).round(4) * 100

#### TransactionAmt

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)

ax1.set_title('Transaction Amounts <= 1000')
ax2.set_title('Transaction Amounts (Log Scale)')

sns.histplot(train_transaction.loc[train_transaction['TransactionAmt'] <= 1000], x='TransactionAmt', bins=100)
sns.histplot(train_transaction, x='TransactionAmt', bins=100, log_scale=True)

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
ax1.set_title('isFraud = 0')
ax2.set_title('isFraud = 1')

sns.histplot(train_transaction.loc[train_transaction['isFraud'] == 0], x='TransactionAmt', color='tab:green', bins=100, log_scale=True, ax=ax1)
sns.histplot(train_transaction.loc[train_transaction['isFraud'] == 1], x='TransactionAmt', color='tab:red', bins=100, log_scale=True, ax=ax2)

plt.tight_layout()
plt.show()

In [ ]:
fraud = train_transaction['isFraud'] == 1

mean_fraud = np.mean(train_transaction.loc[fraud]['TransactionAmt'])
mean_not_fraud = np.mean(train_transaction.loc[not fraud]['TransactionAmt'])

print(f"Valor médio por transações isFraud == 1 é {mean_fraud:.2f}")
print(f"Valor médio por transações isFraud == 0 é {mean_not_fraud:.2f}")

#### Card Issuer

In [ ]:
train_transaction['card4'].value_counts(dropna=False)

In [ ]:
pct_card_fraud = train_transaction.groupby(['card4', 'isFraud'])['TransactionAmt'].sum() / train_transaction.groupby(['card4'])['TransactionAmt'].sum()
pct_card_fraud = (pct_card_fraud * 100).unstack(level=0).reset_index(drop=True)

pct_card_fraud

#### Transaction Type

In [ ]:
train_transaction['card6'].value_counts(dropna=False)

#### DeviceType

In [ ]:
train_identity['DeviceInfo'].unique()

In [ ]:
train_identity['DeviceType'].value_counts(dropna=False)

#### E-mail Domain

In [ ]:
train_transaction['R_emaildomain'].value_counts(dropna=False)

In [ ]:
pct_email_fraud = train_transaction.groupby(['R_emaildomain', 'isFraud'])['TransactionAmt'].sum() / train_transaction.groupby(['R_emaildomain'])['TransactionAmt'].sum()
pct_email_fraud = (pct_email_fraud * 100).unstack('isFraud').dropna()

pct_email_fraud.sort_values(by=1, ascending=False).head(10)

### Análise de Componentes Principais (PCA)

In [ ]:
merged_chunks = []
transaction_chunks = pd.read_csv('../data/train_transaction.csv', chunksize=10 ** 5)

for chunk in transaction_chunks:
    new_chunk = chunk.merge(train_identity, on='TransactionID', how='left')
    merged_chunks.append(new_chunk)

train = pd.concat(merged_chunks)
train = train.groupby('isFraud', group_keys=False)[train.columns].apply(lambda x: x.sample(frac=0.1))

train.shape

In [ ]:
del train_identity, transaction_chunks

In [ ]:
X_train = train.loc[:, train.columns.str.startswith('V')]

print(f'Número de features escolhidas: {X_train.shape[1]}')

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

X_train_transformed = numeric_transformer.fit_transform(X_train)

In [ ]:
pca = PCA()
pca.fit(X_train_transformed)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("n_components")
plt.ylabel("explained_variance_ratio")

plt.show()

In [ ]:
target_variance_ratio = 0.95
d = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= target_variance_ratio) + 1

print(f"Número de dimensões para manter {target_variance_ratio*100}% da variância: {d}")

### Feature Engineering

In [ ]:
train['hasIdentity'] = (train[identity_columns].isna().all(axis=1) == False).astype(int)

adicionar outpu

In [ ]:
amount = 'TransactionAmt'
selected_cols = ['card1', 'card4', 'P_emaildomain', 'R_emaildomain', 'addr1']

for col in selected_cols:
    train[f'amount_mean_{col}'] = train[amount] / train.groupby([col])[amount].transform('mean')
    train[f'amount_std_{col}'] = train[amount] / train.groupby([col])[amount].transform('std')

train['TransactionAmtLog'] = np.log(train[amount])
train['TransactionAmtCents'] = (train[amount] - np.floor(train[amount])).astype(np.float64)

adicionar oupyut

In [ ]:
start_date = datetime.strptime('2022-01-01', '%Y-%m-%d')

train['Date'] = train['TransactionDT'].apply(lambda dt: start_date + timedelta(seconds=dt))

train['Weekday'] = train['Date'].dt.dayofweek
train['Day'] = train['Date'].dt.day
train['Hour'] = train['Date'].dt.hour

train = train.drop(columns=['Date'])

In [ ]:
train[['TransactionDT', 'Weekday', 'Day', 'Hour']].sample()

In [ ]:
train.shape

### Pré-processamento

Categorical Features (Transaction)

- ProductCD
- emaildomain
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

Categorical Features (Identity)

- DeviceType
- DeviceInfo
- id_12 - id_38

Categorical Created Features

- hasIdentity
- Weekday
- Day
- Hour

In [ ]:
CAT_FEATURES = [
    *[f'card{i}' for i in range(1, 7)],
    *[f'M{i}' for i in range(1, 10)],
    *[f'id_{i}' for i in range(12, 39)],
    "ProductCD",
    "emaildomain",
    "addr1", 
    "addr2",
    "P_emaildomain",
    "R_emaildomain",
    "DeviceType",
    "DeviceInfo",
    "hasIdentity",
    "Weekday",
    "Day",
    "Hour",
]

In [ ]:
X = train.drop(columns=['isFraud'])
y = train['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=23)

print(X_train.shape)
print(y_train.shape)

In [ ]:
X_train = X_train.replace([np.inf, -np.inf], -999)
X_test = X_test.replace([np.inf, -np.inf], -999)

In [ ]:
pca_features = X_train.columns[X_train.columns.str.startswith('V')]
cat_features = np.unique(CAT_FEATURES + X_train.select_dtypes(include=['object']).columns.tolist())
num_features = [col for col in X_train.columns if col not in cat_features and col not in pca_features]

print(np.sum(list(map(len, [pca_features, cat_features, num_features]))))

NameError: name 'X_train' is not defined

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('to_string', FunctionTransformer(lambda X: X.astype(str))),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

pca_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=90))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_features),
        ('num', numeric_transformer, num_features),
        ('pca', pca_transformer, pca_features)
    ],
    remainder='passthrough'
)

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

NameError: name 'preprocessor' is not defined

In [ ]:
del X_train, X_test

### Seleção de modelos

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_transformed, y_train)

y_pred = decision_tree.predict(X_test_transformed)

In [ ]:
train_score = decision_tree.score(X_train_transformed, y_train)
test_score = decision_tree.score(X_test_transformed, y_test)

print("Train score: {}".format(train_score))
print("Test score: {}".format(test_score))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')

In [ ]:
results = {}

k_folds = 5
cv = StratifiedKFold(n_splits=k_folds, shuffle=True)

models = {
    'LGBM': LGBMClassifier(verbose=-1, random_state=23),
    'RF': RandomForestClassifier(random_state=23),
    'BC': BaggingClassifier(random_state=23),
    'DT': DecisionTreeClassifier(random_state=23),
}

for name, model in models.items():
    scores = cross_val_score(model, X_train_transformed, y_train, cv=cv, scoring='roc_auc')
    results[name] = scores

    print(f'{name}: {scores.mean()} ({scores.std()})')

In [ ]:
fig = plt.figure() 
fig.suptitle('ROC AUC') 

ax = fig.add_subplot(111) 
ax.set_xticklabels(results.keys()) 

plt.boxplot(results.values()) 

plt.show()

In [ ]:
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)

X_train_res, y_train_res = undersampler.fit_resample(X_train_transformed, y_train)

scores = cross_val_score(RandomForestClassifier(random_state=42), X_train_res, y_train_res, cv=cv, scoring='roc_auc')
print(f'UNDER_RF: {scores.mean()} ({scores.std()})')

results['UNDER_RF'] = scores

In [ ]:
rus_boost = RUSBoostClassifier(random_state=42)

scores = cross_val_score(rus_boost, X_train_transformed, y_train, cv=cv, scoring='roc_auc')
print(f'RUSB: {scores.mean()} ({scores.std()})')

results['RUSB'] = scores

In [ ]:
smote = SMOTE(random_state=42)

X_train_res, y_train_res = smote.fit_resample(X_train_transformed, y_train)

scores = cross_val_score(LGBMClassifier(random_state=42, verbose=-1), X_train_res, y_train_res, cv=cv, scoring='roc_auc')
print(f'SMOTE_LGBM: {scores.mean()} ({scores.std()})')

results['SMOTE_LGBM'] = scores

In [ ]:
sorted_results = dict(sorted(results.items(), key=lambda x: np.median(x[1]), reverse=True))

fig = plt.figure() 
fig.suptitle('ROC AUC') 

ax = fig.add_subplot(111) 
ax.set_xticklabels(sorted_results.keys(), rotation=45) 

plt.boxplot(sorted_results.values()) 
plt.show()

### Treinamento

#### Under Bagging

In [ ]:
clf = RandomForestClassifier(random_state=42)
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)

X_train_res, y_train_res = undersampler.fit_resample(X_train_transformed, y_train)

clf.fit(X_train_res, y_train_res)

y_pred = clf.predict(X_test_transformed)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')

#### SMOTE Bagging

In [ ]:
clf = RandomForestClassifier(random_state=42)
smote = SMOTE(sampling_strategy='minority', random_state=42)

X_train_res, y_train_res = smote.fit_resample(X_train_transformed, y_train)

clf.fit(X_train_res, y_train_res)

y_pred = clf.predict(X_test_transformed)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')

#### Adjusted Weights Random Forest

In [ ]:
clf = RandomForestClassifier(class_weight='balanced', random_state=42)
clf.fit(X_train_transformed, y_train)

y_pred = clf.predict(X_test_transformed)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')

#### GridSearch

In [ ]:
k_folds = 5

param_grid = {
    'n_estimators': [100], # quanto maior melhor
    'max_depth': [50, 60, 75]
}

clf = RandomForestClassifier(random_state=42, n_jobs=8)
cv = StratifiedKFold(n_splits=k_folds, shuffle=True)

In [ ]:
grid_search = GridSearchCV(clf, param_grid, cv=cv, scoring='roc_auc')
grid_search.fit(X_train_res, y_train_res)

print(grid_search.best_score_)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

#### Análise de threshold

In [ ]:
y_scores = cross_val_predict(clf, X_train_res, y_train_res, cv=cv, method='predict_proba')

y_scores

In [ ]:
precision, recall, threshold = precision_recall_curve(y_train_res, y_scores[:, 1])

plt.plot(threshold, precision[:-1], "b--", label="Precision", linewidth=2)
plt.plot(threshold, recall[:-1], "g-", label="Recall", linewidth=2)

plt.xlabel("Threshold")
plt.ylabel("Precision/Recall")
plt.legend(loc="lower left")

plt.show()

In [ ]:
print(f'Target threshold to obtain 90% precision is {threshold[np.argmax(precision >= 0.90)]}')
print(f'Target threshold to obtain 90% precision is {threshold[np.argmin(recall >= 0.90)]}')

In [ ]:
plt.plot(recall, precision, linewidth=2, label="Precision/Recall curve")

In [ ]:
target_threshold = 0.51

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_res, y_train_res)

y_scores = clf.predict_proba(X_test_transformed)[:, 1]
y_pred = (y_scores >= target_threshold).astype(int)

In [ ]:
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')